In [48]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep
from random import randint
import re

In [13]:
df_links = pd.read_csv('../player_link_scrapping/output/player_links.csv')

links = df_links['URL'].tolist()

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

In [37]:
detail_page = '/'.join(links[5].split('/')[:4]) + str('/leistungsdatendetails/') + '/'.join(links[5].split('/')[5:7]) + str('/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1')
detail_page

'https://www.transfermarkt.com/raphael-varane/leistungsdatendetails/spieler/164770/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'

In [78]:
request = rq.get(detail_page,headers=headers)

soup = bs(request.text, 'html.parser')



True

In [79]:
list_of_row_dicts = []

for link in links:

    while True:
            try:
                detail_page = '/'.join(link.split('/')[:4]) + str('/leistungsdatendetails/') + '/'.join(link.split('/')[5:7]) + str('/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1')
    
                request = rq.get(detail_page,headers=headers)
                soup = bs(request.text, 'html.parser')
                rows = soup.find_all('tr', class_=['odd','even'])
           #I need it to check if the player is a goalkeeper, and if not, the follow through with the rest of the code, if it is a keeper, then skip and do not consider for anything     
                if soup.find_all('span', class_='data-header__content')[7].text.strip() == 'Goalkeeper':
                    continue
    
                break
            except IndexError:
                print('Index Error : Sleeping for 10 seconds before retrying')
                sleep(10)
    print(link.split('/')[3])
    sleep(randint(1,3))
    
    for row in rows:

            season = row.find_all('td', class_='zentriert')[0].text

            competition = row.find_all('td', class_='hauptlink no-border-links')[0].text

            club = row.find_all('td', class_='hauptlink no-border-rechts zentriert')[0].find('a')['title']

            called_up = row.find_all('td', class_='zentriert')[2].text

            appearances = row.find_all('td', class_='zentriert')[3].text
            
            points_per_match = row.find_all('td', class_='zentriert')[4].text
            goals = row.find_all('td', class_='zentriert')[5].text
            assists = row.find_all('td', class_='zentriert')[6].text
            own_goals = row.find_all('td', class_='zentriert')[7].text

            subbed_on = row.find_all('td', class_='zentriert')[8].text

            subbed_off = row.find_all('td', class_='zentriert')[9].text

            yellow_cards = row.find_all('td', class_='zentriert')[10].text

            sec_yellow_cards = row.find_all('td', class_='zentriert')[11].text

            red_cards = row.find_all('td', class_='zentriert')[12].text

            penalty_goals = row.find_all('td', class_='zentriert')[13].text

            min_per_goal = row.find_all('td', class_='rechts')[0].text[:-1]

            minutes = row.find_all('td', class_='rechts')[1].text[:-1]

            row_dic = {
            'Player_URL' : link,
            'Season' : season,
            'Competition': competition,
            'Club' : club,
            'Called_Up' : called_up,
            'Appearances' : appearances,
            'Points_Per_Match' : points_per_match,
            'Goals' : goals,
            'Assists' : assists,
            'Own_Goals' : own_goals,
            'Subbed_On' : subbed_on,
            'Subbed_Off' : subbed_off,
            'Yellow_Cards' : yellow_cards,
            'Second_Yellow_Cards' : sec_yellow_cards,
            'Red_Cards' : red_cards,
            'Penalty_Goals' : penalty_goals,
            'Minutes_Per_Goal' : min_per_goal,
            'Minutes_Played' : minutes
            }
            list_of_row_dicts.append(row_dic)

df = pd.DataFrame(list_of_row_dicts)
df = df.loc[df['Season'] != '22/23']

Index Error : Sleeping for 10 seconds before retrying
Index Error : Sleeping for 10 seconds before retrying


KeyboardInterrupt: 

In [214]:
df.to_csv('output/player_season_details.csv')